## Imports

In [1]:
!git clone https://github.com/CCogS-Mx/text-preprocessing.git

fatal: destination path 'text-preprocessing' already exists and is not an empty directory.


In [2]:
import sys 
sys.path.append('Scripts/')
sys.path.append('text-preprocessing/script/')

In [3]:
import os
import pandas as pd
import numpy as np

# Import classes
import text_preprocessing as tp
import feature_extraction as fe
import robust_classic_model as rcm
import cleaning_twitter_data as ctd

# Algoritmos de ML
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split

# Vectorizadores
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qk_le\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qk_le\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\qk_le\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data

In [4]:
training = pd.read_csv('Data/multi_train_labels.csv')
test = pd.read_csv('Data/track2_test_no_labels.csv').drop('Unnamed: 0', axis=1)

In [5]:
prep = tp.Preprocessing('spanish')

In [6]:
training_p = prep.main_preprocess(data=training, 
                                  column='tuit', 
                                  tweet=True, 
                                  tweet_tags=True, 
                                  remove_stop_words=False, 
                                  lemmatize=False, 
                                  translate_emojis=True)

In [7]:
test_p = prep.main_preprocess(data=test, 
                                  column='content', 
                                  tweet=True, 
                                  tweet_tags=True, 
                                  remove_stop_words=False, 
                                  lemmatize=False, 
                                  translate_emojis=True)

In [8]:
training_p.head()

,tuit,G,L,B,T,O
0,bueno pues ya paso la euforia papa ahora si di...,1,0,0,0,0
1,esos de bts se me hacen seres asexuales eunuco...,0,0,0,0,1
2,la partida de skywars que me ha hecho enojar m...,1,0,0,0,0
3,se te acabo tu juego maldito joto asqueroso es...,1,0,0,0,0
4,sera a los jotitos ellos prefieren las banderi...,1,0,0,0,0


## Improving best classifiers with training data only

In [9]:
from sklearn.metrics import f1_score
from sklearn.calibration import CalibratedClassifierCV

In [10]:
vectorizer = CountVectorizer(ngram_range = (1,1), lowercase = False)
oth_feats_vectorizer = CountVectorizer(ngram_range = (1,1), lowercase = False)
labels = ['G', 'L', 'B', 'T', 'O']

### Unigramas

In [11]:
models_list_count = [[LogisticRegression(C=1000.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=100, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=10.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=1000.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=50, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=10.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=0.001, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=10, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=0.001, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=1000.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=50, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=1.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=1000.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=50, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=1.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')]]

models_list_tfidf = [[LogisticRegression(C=1000.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=50, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=100.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=1000.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=50, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=1000.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=0.001, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=10, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=0.001, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=1000.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=10, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=100.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=1000.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=50, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=10.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')]]

model_names = ['Logistic Regression', 'Random Forest', 'Linear SVC', 'KNN']

In [19]:
for i, label in enumerate(labels):
    print(f'Best hyperparams for: {label}')

    # Split
    X_train, X_test, y_train, y_test = train_test_split(training_p, 
                                                    training_p[label], 
                                                    test_size=0.05, 
                                                    random_state=42)
    # Train
    f_train = fe.FeatureExtraction(data = X_train, 
                                    text_column = 'tuit', 
                                    lemma = True, 
                                    pos = True, 
                                    tag = True,                                
                                    other_features = True,
                                    vectorizer = vectorizer,
                                    oth_feats_vectorizer = oth_feats_vectorizer,
                                    language = 'es')

    X = f_train.add_features_to_test()
    y_train = y_train.tolist()

    # Test
    f_train = fe.FeatureExtraction(data = X_test, 
                                    text_column = 'tuit', 
                                    lemma = True, 
                                    pos = True, 
                                    tag = True,                                
                                    other_features = True,
                                    vectorizer = vectorizer,
                                    oth_feats_vectorizer = oth_feats_vectorizer,
                                    language = 'es')

    x = f_train.add_features_to_test()
    y_test = y_test.tolist()

    # Models
    models_count = models_list_count[i]
    models_tfidf = models_list_tfidf[i]

    # Finding best hyperparams
    for j in range(2):
        if j == 0:
            print('Count vectorizer')
            for k, model in enumerate(models_count):
                count_v = CountVectorizer(ngram_range = (1,1), lowercase = False)
                X_Train = count_v.fit_transform(X['tuit'])
                X_Test = count_v.transform(x['tuit'])
                model.fit(X_Train, y_train)
                y_pred = model.predict(X_Test)
                print(f'Model: {model_names[k]}, f1: {round(f1_score(y_test, y_pred, average="macro"), 4)}')
        else:
            print('TFIDF vectorizer')
            for k, model in enumerate(models_tfidf):
                count_v = TfidfVectorizer(ngram_range = (1,1), lowercase = False)
                X_Train = count_v.fit_transform(X['tuit'])
                X_Test = count_v.transform(x['tuit'])
                model.fit(X_Train, y_train)
                y_pred = model.predict(X_Test)
                print(f'Model: {model_names[k]}, f1: {round(f1_score(y_test, y_pred, average="macro"), 4)}')
    
    print('')


Best hyperparams for: G
Count vectorizer
Model: Logistic Regression, f1: 1.0
Model: Random Forest, f1: 0.8322
Model: Linear SVC, f1: 1.0
Model: KNN, f1: 0.4966
TFIDF vectorizer
Model: Logistic Regression, f1: 0.9062
Model: Random Forest, f1: 0.7905
Model: Linear SVC, f1: 0.9062
Model: KNN, f1: 0.4054

Best hyperparams for: L
Count vectorizer
Model: Logistic Regression, f1: 0.6423
Model: Random Forest, f1: 0.4699
Model: Linear SVC, f1: 0.6423
Model: KNN, f1: 0.4634
TFIDF vectorizer
Model: Logistic Regression, f1: 0.6423
Model: Random Forest, f1: 0.4699
Model: Linear SVC, f1: 0.6423
Model: KNN, f1: 0.4699

Best hyperparams for: B
Count vectorizer
Model: Logistic Regression, f1: 0.4943
Model: Random Forest, f1: 0.4943
Model: Linear SVC, f1: 0.4943
Model: KNN, f1: 0.4943
TFIDF vectorizer
Model: Logistic Regression, f1: 0.4943
Model: Random Forest, f1: 0.4943
Model: Linear SVC, f1: 0.4943
Model: KNN, f1: 0.4943

Best hyperparams for: T
Count vectorizer
Model: Logistic Regression, f1: 0.7744

### Unigramas + bigramas

In [20]:
models_list_count = [[LogisticRegression(C=1000.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=100, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=100.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=1000.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=50, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=10.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=0.001, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=10, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=0.001, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=100.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=10, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=1.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=10.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=50, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=0.1, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')]]

models_list_tfidf = [[LogisticRegression(C=100.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=100, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=100.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=10.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=50, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=10.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=0.001, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=10, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=0.001, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=100.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=50, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=100.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=10.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=50, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=10.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')]]

model_names = ['Logistic Regression', 'Random Forest', 'Linear SVC', 'KNN']

In [21]:
for i, label in enumerate(labels):
    print(f'Best hyperparams for: {label}')

    # Split
    X_train, X_test, y_train, y_test = train_test_split(training_p, 
                                                    training_p[label], 
                                                    test_size=0.05, 
                                                    random_state=42)
    # Train
    f_train = fe.FeatureExtraction(data = X_train, 
                                    text_column = 'tuit', 
                                    lemma = True, 
                                    pos = True, 
                                    tag = True,                                
                                    other_features = True,
                                    vectorizer = vectorizer,
                                    oth_feats_vectorizer = oth_feats_vectorizer,
                                    language = 'es')

    X = f_train.add_features_to_test()
    y_train = y_train.tolist()

    # Test
    f_train = fe.FeatureExtraction(data = X_test, 
                                    text_column = 'tuit', 
                                    lemma = True, 
                                    pos = True, 
                                    tag = True,                                
                                    other_features = True,
                                    vectorizer = vectorizer,
                                    oth_feats_vectorizer = oth_feats_vectorizer,
                                    language = 'es')

    x = f_train.add_features_to_test()
    y_test = y_test.tolist()

    # Models
    models_count = models_list_count[i]
    models_tfidf = models_list_tfidf[i]

    # Finding best hyperparams
    for j in range(2):
        if j == 0:
            print('Count vectorizer')
            for k, model in enumerate(models_count):
                count_v = CountVectorizer(ngram_range = (1,2), lowercase = False)
                X_Train = count_v.fit_transform(X['tuit'])
                X_Test = count_v.transform(x['tuit'])
                model.fit(X_Train, y_train)
                y_pred = model.predict(X_Test)
                print(f'Model: {model_names[k]}, f1: {round(f1_score(y_test, y_pred, average="macro"), 4)}')
        else:
            print('TFIDF vectorizer')
            for k, model in enumerate(models_tfidf):
                count_v = TfidfVectorizer(ngram_range = (1,2), lowercase = False)
                X_Train = count_v.fit_transform(X['tuit'])
                X_Test = count_v.transform(x['tuit'])
                model.fit(X_Train, y_train)
                y_pred = model.predict(X_Test)
                print(f'Model: {model_names[k]}, f1: {round(f1_score(y_test, y_pred, average="macro"), 4)}')
    
    print('')


Best hyperparams for: G
Count vectorizer
Model: Logistic Regression, f1: 0.6944
Model: Random Forest, f1: 0.5753
Model: Linear SVC, f1: 0.5753
Model: KNN, f1: 0.4532
TFIDF vectorizer
Model: Logistic Regression, f1: 0.6384
Model: Random Forest, f1: 0.6181
Model: Linear SVC, f1: 0.6384
Model: KNN, f1: 0.4776

Best hyperparams for: L
Count vectorizer
Model: Logistic Regression, f1: 0.6423
Model: Random Forest, f1: 0.4699
Model: Linear SVC, f1: 0.4699
Model: KNN, f1: 0.4699
TFIDF vectorizer
Model: Logistic Regression, f1: 0.4699
Model: Random Forest, f1: 0.4699
Model: Linear SVC, f1: 0.4699
Model: KNN, f1: 0.4634

Best hyperparams for: B
Count vectorizer
Model: Logistic Regression, f1: 0.4943
Model: Random Forest, f1: 0.4943
Model: Linear SVC, f1: 0.4943
Model: KNN, f1: 0.4943
TFIDF vectorizer
Model: Logistic Regression, f1: 0.4943
Model: Random Forest, f1: 0.4943
Model: Linear SVC, f1: 0.4943
Model: KNN, f1: 0.4884

Best hyperparams for: T
Count vectorizer
Model: Logistic Regression, f1: 

### Unigramas, bigramas y trigramas

In [22]:
models_list_count = [[LogisticRegression(C=1000.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=100, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=10.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=100.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=10, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=0.1, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=0.001, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=10, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=0.001, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=100.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=10, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=0.1, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=1.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=50, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=1.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')]]

models_list_tfidf = [[LogisticRegression(C=100.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=100, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=10.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=0.001, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=10, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=0.001, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=0.001, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=10, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=0.001, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=10.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=10, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=100.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')],
                    [LogisticRegression(C=10.0, max_iter=1000, penalty='l2', random_state=42), 
                    RandomForestClassifier(max_depth=50, n_estimators=10,random_state=42), 
                    CalibratedClassifierCV(LinearSVC(C=1.0, max_iter=10000, penalty='l2', random_state=42)), 
                    KNeighborsClassifier(n_neighbors=3, weights='distance')]]

model_names = ['Logistic Regression', 'Random Forest', 'Linear SVC', 'KNN']

In [23]:
for i, label in enumerate(labels):
    print(f'Best hyperparams for: {label}')

    # Split
    X_train, X_test, y_train, y_test = train_test_split(training_p, 
                                                    training_p[label], 
                                                    test_size=0.05, 
                                                    random_state=42)
    # Train
    f_train = fe.FeatureExtraction(data = X_train, 
                                    text_column = 'tuit', 
                                    lemma = True, 
                                    pos = True, 
                                    tag = True,                                
                                    other_features = True,
                                    vectorizer = vectorizer,
                                    oth_feats_vectorizer = oth_feats_vectorizer,
                                    language = 'es')

    X = f_train.add_features_to_test()
    y_train = y_train.tolist()

    # Test
    f_train = fe.FeatureExtraction(data = X_test, 
                                    text_column = 'tuit', 
                                    lemma = True, 
                                    pos = True, 
                                    tag = True,                                
                                    other_features = True,
                                    vectorizer = vectorizer,
                                    oth_feats_vectorizer = oth_feats_vectorizer,
                                    language = 'es')

    x = f_train.add_features_to_test()
    y_test = y_test.tolist()

    # Models
    models_count = models_list_count[i]
    models_tfidf = models_list_tfidf[i]

    # Finding best hyperparams
    for j in range(2):
        if j == 0:
            print('Count vectorizer')
            for k, model in enumerate(models_count):
                count_v = CountVectorizer(ngram_range = (1,3), lowercase = False)
                X_Train = count_v.fit_transform(X['tuit'])
                X_Test = count_v.transform(x['tuit'])
                model.fit(X_Train, y_train)
                y_pred = model.predict(X_Test)
                print(f'Model: {model_names[k]}, f1: {round(f1_score(y_test, y_pred, average="macro"), 4)}')
        else:
            print('TFIDF vectorizer')
            for k, model in enumerate(models_tfidf):
                count_v = TfidfVectorizer(ngram_range = (1,3), lowercase = False)
                X_Train = count_v.fit_transform(X['tuit'])
                X_Test = count_v.transform(x['tuit'])
                model.fit(X_Train, y_train)
                y_pred = model.predict(X_Test)
                print(f'Model: {model_names[k]}, f1: {round(f1_score(y_test, y_pred, average="macro"), 4)}')
    
    print('')


Best hyperparams for: G
Count vectorizer
Model: Logistic Regression, f1: 0.5417
Model: Random Forest, f1: 0.5036
Model: Linear SVC, f1: 0.5036
Model: KNN, f1: 0.4903
TFIDF vectorizer
Model: Logistic Regression, f1: 0.5036
Model: Random Forest, f1: 0.4903
Model: Linear SVC, f1: 0.5753
Model: KNN, f1: 0.6181

Best hyperparams for: L
Count vectorizer
Model: Logistic Regression, f1: 0.4634
Model: Random Forest, f1: 0.4699
Model: Linear SVC, f1: 0.4699
Model: KNN, f1: 0.4699
TFIDF vectorizer
Model: Logistic Regression, f1: 0.4699
Model: Random Forest, f1: 0.4699
Model: Linear SVC, f1: 0.4699
Model: KNN, f1: 0.4568

Best hyperparams for: B
Count vectorizer
Model: Logistic Regression, f1: 0.4943
Model: Random Forest, f1: 0.4943
Model: Linear SVC, f1: 0.4943
Model: KNN, f1: 0.4943
TFIDF vectorizer
Model: Logistic Regression, f1: 0.4943
Model: Random Forest, f1: 0.4943
Model: Linear SVC, f1: 0.4943
Model: KNN, f1: 0.4884

Best hyperparams for: T
Count vectorizer
Model: Logistic Regression, f1: 